We prepared the true labels y by grouping the ICD9 codes into 283 groups using CCS single-level diagnosis grouper. <br>
https://www.hcup-us.ahrq.gov/toolssoftware/ccs/AppendixASingleDX.txt

In [1]:
import pickle
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [2]:
f = open('./AppendixASingleDX.txt')
groups = f.read().split('\n\n')[2:]
groups

['1    Tuberculosis\n     01000 01001 01002 01003 01004 01005 01006 01010 01011 01012 01013 01014 01015 01016 01080 01081 01082 01083 01084 01085\n     01086 01090 01091 01092 01093 01094 01095 01096 01100 01101 01102 01103 01104 01105 01106 01110 01111 01112 01113 01114\n     01115 01116 01120 01121 01122 01123 01124 01125 01126 01130 01131 01132 01133 01134 01135 01136 01140 01141 01142 01143\n     01144 01145 01146 01150 01151 01152 01153 01154 01155 01156 01160 01161 01162 01163 01164 01165 01166 01170 01171 01172\n     01173 01174 01175 01176 01180 01181 01182 01183 01184 01185 01186 01190 01191 01192 01193 01194 01195 01196 01200 01201\n     01202 01203 01204 01205 01206 01210 01211 01212 01213 01214 01215 01216 01220 01221 01222 01223 01224 01225 01226 01230\n     01231 01232 01233 01234 01235 01236 01280 01281 01282 01283 01284 01285 01286 01300 01301 01302 01303 01304 01305 01306\n     01310 01311 01312 01313 01314 01315 01316 01320 01321 01322 01323 01324 01325 01326 01330 01

In [3]:
CCS = {}
for group in groups:
    key = re.split('\s',group)[0]
    codeList = []
    lines = re.split('\n',group)[1:]
    for line in lines:
        for i in re.split('\s',line):
            if i != '':
                codeList.append(i)
    CCS[key] = codeList
CCS

{'1': ['01000',
  '01001',
  '01002',
  '01003',
  '01004',
  '01005',
  '01006',
  '01010',
  '01011',
  '01012',
  '01013',
  '01014',
  '01015',
  '01016',
  '01080',
  '01081',
  '01082',
  '01083',
  '01084',
  '01085',
  '01086',
  '01090',
  '01091',
  '01092',
  '01093',
  '01094',
  '01095',
  '01096',
  '01100',
  '01101',
  '01102',
  '01103',
  '01104',
  '01105',
  '01106',
  '01110',
  '01111',
  '01112',
  '01113',
  '01114',
  '01115',
  '01116',
  '01120',
  '01121',
  '01122',
  '01123',
  '01124',
  '01125',
  '01126',
  '01130',
  '01131',
  '01132',
  '01133',
  '01134',
  '01135',
  '01136',
  '01140',
  '01141',
  '01142',
  '01143',
  '01144',
  '01145',
  '01146',
  '01150',
  '01151',
  '01152',
  '01153',
  '01154',
  '01155',
  '01156',
  '01160',
  '01161',
  '01162',
  '01163',
  '01164',
  '01165',
  '01166',
  '01170',
  '01171',
  '01172',
  '01173',
  '01174',
  '01175',
  '01176',
  '01180',
  '01181',
  '01182',
  '01183',
  '01184',
  '01185',
  '01

In [4]:
pickle.dump(CCS, open('CCS', 'wb'), -1)

In [5]:
# 映射pid-vid、vid-日期
pidAdmMap = {}
admDateMap = {}
infd = open('ADMISSIONS.csv', 'r')
infd.readline()
for line in infd:
    tokens = line.strip().split(',')
    pid = int(tokens[1])   # SUBJECT_ID
    admId = int(tokens[2]) # HADM_ID
    admTime = datetime.strptime(tokens[3], '%Y-%m-%d %H:%M:%S')
    admDateMap[admId] = admTime
    if pid in pidAdmMap: 
        pidAdmMap[pid].append(admId)
    else: 
        pidAdmMap[pid] = [admId]
infd.close()
pidAdmMap

{22: [165315],
 23: [152223, 124321],
 24: [161859],
 25: [129635],
 26: [197661],
 27: [134931],
 28: [162569],
 30: [104557],
 31: [128652],
 32: [175413],
 33: [176176],
 34: [115799, 144319],
 35: [166707],
 36: [182104, 122659, 165660],
 37: [188670],
 38: [185910],
 39: [106266],
 41: [101757],
 357: [174486, 145674, 122609, 101651, 117876],
 358: [110872],
 359: [144265],
 360: [154871],
 361: [108205, 148959],
 362: [142749, 150873],
 363: [196503],
 364: [136153],
 365: [139873],
 366: [134462],
 367: [119940],
 368: [105889, 138061],
 369: [145787],
 370: [123421],
 371: [113500],
 372: [144383],
 42: [119203],
 43: [146828],
 44: [181750],
 45: [157907],
 46: [144073],
 49: [190539],
 50: [132761],
 51: [196010],
 52: [190797],
 53: [155385],
 54: [138795],
 55: [190665],
 56: [181711],
 57: [162774],
 58: [177285],
 59: [104130],
 60: [198892],
 61: [176332, 189535],
 62: [116009],
 63: [195961],
 64: [172056],
 65: [143430],
 66: [104518],
 67: [186474, 155252],
 68: [1704

In [6]:
# 映射vid-诊断编码
admDxMap = {}
infd = open('DIAGNOSES_ICD.csv', 'r')
infd.readline()
for line in infd:
    tokens = line.strip().split(',')
    admId = int(tokens[2]) 
    dxStr = tokens[4][1:-1]
    if admId in admDxMap: 
        admDxMap[admId].append(dxStr)
    else: 
        admDxMap[admId] = [dxStr]
infd.close()
admDxMap

{172335: ['40301',
  '486',
  '58281',
  '5855',
  '4254',
  '2762',
  '7100',
  '2767',
  '7243',
  '45829',
  '2875',
  '28521',
  '28529',
  '27541'],
 173633: ['40301',
  '5856',
  '58381',
  '7100',
  '5589',
  '2875',
  '28521',
  '45829',
  '32723',
  '22804',
  '33829',
  '78900',
  '79092',
  'V4511',
  'V1251'],
 174105: ['53100', '41071', '2859', '41401', '725'],
 109976: ['1915', '3314', '53081'],
 178393: ['41401', '4111', '48283', '2859', '2720', '3051'],
 114585: ['1940',
  '1977',
  '2553',
  '4240',
  '5845',
  '99859',
  '6822',
  '5119',
  '5990',
  '4280',
  '34982',
  '4019',
  'V1000',
  'V453',
  'V5865',
  '0413',
  '2518',
  'E9320'],
 127203: ['V3001', 'V053', 'V290'],
 140784: ['5715',
  '7895',
  '07054',
  '2875',
  '4280',
  '2851',
  '2765',
  '4019',
  '25000'],
 164853: ['570',
  '07044',
  '5712',
  '5849',
  '5724',
  '5118',
  '2867',
  '51882',
  '2875',
  '0389',
  '486',
  '2800',
  '2639',
  '2761',
  '2767',
  '99592',
  '30393',
  '4019',
  '57

In [7]:
# 映射vid-CCS
admDxMap_CCS = {}
infd = open('DIAGNOSES_ICD.csv', 'r')
infd.readline()
for line in infd:
    tokens = line.strip().split(',')
    admId = int(tokens[2]) 
    dxStr = tokens[4][1:-1]
    for group, codeList in CCS.items():
        for code in codeList:
            if code == dxStr:
                groupStr = int(group)
                break
    if admId in admDxMap_CCS: 
        admDxMap_CCS[admId].append(groupStr)
    else: 
        admDxMap_CCS[admId] = [groupStr]
infd.close()
admDxMap_CCS

{172335: [99, 122, 156, 158, 97, 55, 210, 55, 205, 238, 62, 59, 59, 58],
 173633: [99,
  158,
  156,
  210,
  154,
  62,
  59,
  238,
  259,
  47,
  95,
  251,
  259,
  158,
  118],
 174105: [153, 100, 59, 101, 211],
 109976: [35, 81, 138],
 178393: [101, 101, 122, 59, 53, 663],
 114585: [41,
  42,
  51,
  96,
  157,
  238,
  197,
  130,
  159,
  108,
  95,
  98,
  18,
  155,
  257,
  3,
  51,
  2617],
 127203: [218, 10, 256],
 140784: [151, 151, 6, 62, 108, 60, 55, 98, 49],
 164853: [151,
  6,
  660,
  157,
  151,
  130,
  62,
  131,
  62,
  2,
  122,
  59,
  52,
  55,
  55,
  2,
  660,
  98,
  151,
  49,
  48,
  4],
 147035: [218, 10],
 135738: [236, 234, 234, 244, 244, 244, 662, 661],
 188923: [218, 10, 256],
 135750: [218, 213, 224],
 195632: [100, 101, 98, 49, 663],
 112906: [115, 122, 127, 6, 117, 237, 101, 98, 53],
 134369: [114, 157, 158, 159, 238, 114, 99, 54, 127, 238, 106, 29, 19, 8, 3],
 138376: [155,
  145,
  127,
  238,
  2,
  2,
  238,
  122,
  157,
  158,
  52,
  108,
 

In [8]:
# 映射pid-(日期+CCS)
pidSeqMap = {}
for pid, admIdList in pidAdmMap.items():
    if len(admIdList) < 2: 
        continue

    sortedList = sorted([(admDateMap[admId], admDxMap_CCS[admId]) for admId in admIdList])
    pidSeqMap[pid] = sortedList

pidSeqMap

{23: [(datetime.datetime(2153, 9, 3, 7, 15),
   [101, 101, 96, 101, 53, 98, 164, 94]),
  (datetime.datetime(2157, 10, 18, 19, 34),
   [47, 95, 83, 96, 98, 53, 53, 101, 259, 663])],
 34: [(datetime.datetime(2186, 7, 18, 16, 46),
   [100, 108, 97, 106, 238, 105, 101, 2616]),
  (datetime.datetime(2191, 2, 23, 5, 23),
   [106, 108, 105, 101, 257, 108, 48, 88])],
 36: [(datetime.datetime(2131, 4, 30, 7, 15),
   [101, 101, 127, 98, 663, 138, 164, 32, 162]),
  (datetime.datetime(2131, 5, 12, 19, 49),
   [238, 238, 238, 118, 237, 127, 101, 101, 98, 164, 138, 32]),
  (datetime.datetime(2134, 5, 10, 11, 30),
   [143, 238, 131, 122, 238, 130, 101, 127, 98, 164, 651, 32])],
 357: [(datetime.datetime(2197, 12, 6, 7, 13),
   [151, 151, 62, 153, 83, 122, 131, 108, 151, 259, 49, 257, 151]),
  (datetime.datetime(2198, 8, 2, 4, 49),
   [83,
    2,
    2,
    131,
    159,
    151,
    16,
    157,
    151,
    106,
    211,
    249,
    122,
    49,
    48,
    98,
    238,
    59,
    126]),
  (datetim

In [9]:
# 创建pid、日期[[date1,date2], ...]、CCS[[[c1,c2],[c3]],...]
pids = []
dates = []
seqs = []
for pid, visits in pidSeqMap.items():
    pids.append(pid)
    seq = []
    date = []
    for visit in visits:
        date.append(visit[0])
        seq.append(visit[1])
    dates.append(date)
    seqs.append(seq)
seqs

[[[101, 101, 96, 101, 53, 98, 164, 94],
  [47, 95, 83, 96, 98, 53, 53, 101, 259, 663]],
 [[100, 108, 97, 106, 238, 105, 101, 2616],
  [106, 108, 105, 101, 257, 108, 48, 88]],
 [[101, 101, 127, 98, 663, 138, 164, 32, 162],
  [238, 238, 238, 118, 237, 127, 101, 101, 98, 164, 138, 32],
  [143, 238, 131, 122, 238, 130, 101, 127, 98, 164, 651, 32]],
 [[151, 151, 62, 153, 83, 122, 131, 108, 151, 259, 49, 257, 151],
  [83,
   2,
   2,
   131,
   159,
   151,
   16,
   157,
   151,
   106,
   211,
   249,
   122,
   49,
   48,
   98,
   238,
   59,
   126],
  [55,
   249,
   157,
   2,
   2,
   83,
   131,
   50,
   199,
   153,
   16,
   151,
   52,
   108,
   108,
   138,
   55,
   134,
   98,
   53,
   101,
   143,
   59,
   155],
  [129,
   108,
   83,
   55,
   16,
   159,
   199,
   151,
   108,
   199,
   59,
   151,
   49,
   48,
   98,
   101,
   155],
  [122,
   83,
   151,
   131,
   199,
   151,
   130,
   16,
   55,
   108,
   199,
   108,
   50,
   59,
   151,
   98,
   101,
   4

In [13]:
# 映射数字-诊断编码
types = {}
newSeqs = []
for patient in seqs: # patient = [[c1,c2],[c3]]
    newPatient = []
    for visit in patient: # visit = [c1,c2]
        newVisit = []
        for code in visit: # code = c1
            if code in types:
                newVisit.append(types[code])
            else:
                types[code] = len(types)
                newVisit.append(types[code]) # newVisit: 转化为数字后每次就诊的诊断
        newPatient.append(newVisit) # newPatient: 患者就诊记录的集合
    newSeqs.append(newPatient)
newSeqs

[[[0, 0, 1, 0, 2, 3, 4, 5], [6, 7, 8, 1, 3, 2, 2, 0, 9, 10]],
 [[11, 12, 13, 14, 15, 16, 0, 17], [14, 12, 16, 0, 18, 12, 19, 20]],
 [[0, 0, 21, 3, 10, 22, 4, 23, 24],
  [15, 15, 15, 25, 26, 21, 0, 0, 3, 4, 22, 23],
  [27, 15, 28, 29, 15, 30, 0, 21, 3, 4, 31, 23]],
 [[32, 32, 33, 34, 8, 29, 28, 12, 32, 9, 35, 18, 32],
  [8, 36, 36, 28, 37, 32, 38, 39, 32, 14, 40, 41, 29, 35, 19, 3, 15, 42, 43],
  [44,
   41,
   39,
   36,
   36,
   8,
   28,
   45,
   46,
   34,
   38,
   32,
   47,
   12,
   12,
   22,
   44,
   48,
   3,
   2,
   0,
   27,
   42,
   49],
  [50, 12, 8, 44, 38, 37, 46, 32, 12, 46, 42, 32, 35, 19, 3, 0, 49],
  [29,
   8,
   32,
   28,
   46,
   32,
   30,
   38,
   44,
   12,
   46,
   12,
   45,
   42,
   32,
   3,
   0,
   19,
   42,
   51,
   52,
   49]],
 [[26, 39, 53, 54, 44, 44, 17, 55, 56, 2],
  [25, 34, 53, 15, 57, 26, 58, 3, 2, 35, 59, 6, 14]],
 [[60, 14, 1, 1, 37, 61, 3, 62], [60, 12, 14, 1, 3, 62]],
 [[29, 21, 11, 39, 53, 51, 42, 22, 31],
  [11, 29, 28, 12, 44

In [14]:
pickle.dump(newSeqs, open('CCS.seqs', 'wb'), -1)

In [ ]:
# newseqs = [[[int(code) for code in visit] for visit in patient] for patient in seqs]
# newseqs